In [1]:
import numpy as np
import pandas as pd
import keras

/home/chyld/.local/apps/miniconda3/envs/gamma/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, SpatialDropout1D
from keras.layers import LSTM
from keras.datasets import imdb

In [49]:
max_features = 5000
maxlen = 80
batch_size = 32

In [50]:
# http://ai.stanford.edu/~amaas/data/sentiment/
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

In [51]:
# number of reviews
len(x_train)

25000

In [52]:
# words in each review
for i in range(10):
    print(i, len(x_train[i]))

0 218
1 189
2 141
3 550
4 147
5 43
6 123
7 562
8 233
9 130


In [53]:
# sentiment, positive or negative
np.unique(y_train)

array([0, 1])

In [76]:
INDEX_FROM = -5000
word_to_id = imdb.get_word_index()
id_to_word = {v:k for k,v in word_to_id.items()}

# word_to_id = {k:(v+INDEX_FROM) for k,v in word_to_id.items()}
# word_to_id["<PAD>"] = 0
# word_to_id["<START>"] = 1
# word_to_id["<UNK>"] = 2

In [75]:
word_to_id['this']

11

In [79]:
id_to_word[11]

'this'

In [85]:
for idx in x_train[0]:
    val = idx - 3
    if val < 0:
        val = 1
    print(id_to_word[val])

the
this
film
was
just
brilliant
casting
location
scenery
story
direction
everyone's
really
suited
the
part
they
played
and
you
could
just
imagine
being
there
robert
the
is
an
amazing
actor
and
now
the
same
being
director
the
father
came
from
the
same
scottish
island
as
myself
so
i
loved
the
fact
there
was
a
real
connection
with
this
film
the
witty
remarks
throughout
the
film
were
great
it
was
just
brilliant
so
much
that
i
bought
the
film
as
soon
as
it
was
released
for
the
and
would
recommend
it
to
everyone
to
watch
and
the
fly
the
was
amazing
really
cried
at
the
end
it
was
so
sad
and
you
know
what
they
say
if
you
cry
at
a
film
it
must
have
been
good
and
this
definitely
was
also
the
to
the
two
little
the
that
played
the
the
of
norman
and
paul
they
were
just
brilliant
children
are
often
left
out
of
the
the
list
i
think
because
the
stars
that
play
them
all
grown
up
are
such
a
big
the
for
the
whole
film
but
these
children
are
amazing
and
should
be
the
for
what
they
have
done
don't
you
thi

In [66]:
id_to_word = {value:key for key,value in word_to_id.items()}

In [67]:
' '.join([id_to_word[idx] for idx in x_train[0]])

'<START> whoopi correctly heights retired hyped bridges crippled sale merits additional surpasses directions denver depicting furious integrity remakes kissing wholly marion retired pete grabs sid height <UNK> pin peters skits breed kissing tube depicting ordered grabs milk <UNK> charged cancer yard depicting ordered vengeful 1994 spock bunny biography swim option depicting premiere sid heights tooth inability excruciating abraham whoopi correctly depicting seductive ing iv depicting correctly metaphor eugene assumed heights retired hyped biography loyalty domino swim homosexuality depicting correctly spock beware spock assumed heights demonstrates inevitably <UNK> kissing 1971 fantasies assumed kubrick posters kubrick dee kissing depicting denouement <UNK> heights skits directions stripper valley depicting disliked assumed heights biography nicolas kissing wholly confrontation compassion integrity \x85 meryl wholly lester valley tooth correctly assumed stumbled othello deaf dreck kiss

In [47]:
id_to_word[2]

'and'

In [10]:
# make all rows the same length
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

In [11]:
x_train.shape

(25000, 80)

In [24]:
model = Sequential()
model.add(Embedding(max_features, 4))
# model.add(SpatialDropout1D(0.5))
model.add(LSTM(4, dropout=0.5, recurrent_dropout=0.5))
model.add(Dense(1, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 4)           80000     
_________________________________________________________________
lstm_3 (LSTM)                (None, 4)                 144       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 5         
Total params: 80,149
Trainable params: 80,149
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=1,
          validation_data=(x_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/1
25000/25000 [==============================] - 38s 2ms/step - loss: 0.6232 - acc: 0.6595 - val_loss: 0.5444 - val_acc: 0.7502


In [26]:
score, acc = model.evaluate(x_test, y_test, batch_size=batch_size)

25000/25000 [==============================] - 4s 180us/step


In [27]:
score, acc

(0.5444021554374695, 0.75024)

In [28]:
embedding = model.get_layer('embedding_2').get_weights()
embedding

ValueError: No such layer: embedding_2

In [ ]:
embedding[0].shape

In [ ]:
#predict sentiment from reviews
bad = "this movie was terrible and bad"
good = "i really liked the movie and had fun"
for review in [good,bad]:
    tmp = []
    for word in review.split(" "):
        tmp.append(word_to_id[word])
    tmp_padded = sequence.pad_sequences([tmp], maxlen=max_review_length) 
    print("%s. Sentiment: %s" % (review,model.predict(array([tmp_padded][0]))[0][0]))
i really liked the movie and had fun. Sentiment: 0.715537
this movie was terrible and bad. Sentiment: 0.0353295